# MPh API 全覆盖测试（基于 tritum_cycle_Model.mph）

本笔记本专注于对 MPh 核心 API 进行系统性测试。
执行前请确保 COMSOL 可用且许可有效。

In [1]:
import mph
import os
from pprint import pprint

# 启动 Comsol 客户端
try:
    client = mph.start()
    print(f"MPh {mph.__version__} connected to Comsol {client.version}")
except Exception as e:
    raise RuntimeError(f"Connection failed: {e}")

# 自动查找并加载模型
model_name = 'tritum_cycle_Model.mph'
search_paths = [
    model_name,
    os.path.join('..', 'experiment_00', model_name),
    r'D:\\Administrator\\Documents\\Github\\tricys_mph\\experiment_00\\tritum_cycle_Model.mph',
]

model = None
for p in search_paths:
    if os.path.exists(p):
        print(f"Loading model from: {p} ...")
        model = client.load(p)
        break

if not model:
    raise FileNotFoundError(f"无法找到 {model_name}，请检查路径。")

print("Model loaded successfully.")

MPh 1.2.4 connected to Comsol 6.2
Loading model from: tritum_cycle_Model.mph ...
Model loaded successfully.


## 1. MPh API 全覆盖测试（基于 tritum_cycle_Model.mph）

以下小节覆盖配置、客户端、模型、节点、参数、评估与清理等核心 API。

In [2]:
# 1.1 配置 API：mph.config.option
print("Current options:")
pprint(mph.config.option(None, None))

# 示例：临时启用缓存（可按需关闭）
mph.config.option('caching', True)
print("Caching enabled:", mph.config.option('caching', None))

Current options:
{'caching': False, 'classkit': False, 'session': 'platform-dependent'}
Caching enabled: True


In [3]:
# 1.2 Client API：会话信息、已加载模型、许可模块
print(client)
print("Standalone:", client.standalone)
print("Version:", client.version)
print("Loaded models:", client.names())
print("Licensed modules (subset):", client.modules()[:10])

Client(stand-alone)
Standalone: True
Version: 6.2
Loaded models: ['tritum_cycle_Model']
Licensed modules (subset): ['Comsol core', 'AC/DC', 'Acoustics', 'Battery Design', 'CAD Import', 'CFD', 'Chemical Reaction Engineering', 'Cluster Computing', 'Composite Materials', 'Corrosion']


In [4]:
# 1.3 Model API：基础信息与问题检测
print("Model name:", model.name())
print("Model file:", model.file())
print("Model version:", model.version())
print("Model modules:", model.modules())

problems = model.problems()
print("Problems:", "None" if not problems else f"{len(problems)} issue(s)")
if problems:
    print(problems[0])

Model name: tritum_cycle_Model
Model file: D:\Administrator\Documents\Github\tricys_mph\experiment_05\tritum_cycle_Model.mph
Model version: 6.2
Model modules: ['Comsol core']
Problems: None


In [5]:
# 1.4 参数读写 API（带恢复）
original_tbr = model.parameter('TBR')
print("Original TBR:", original_tbr)

model.parameter('TBR', '1.15')
print("Updated TBR:", model.parameter('TBR'))

# 恢复原值
model.parameter('TBR', original_tbr)
print("Restored TBR:", model.parameter('TBR'))

# 批量读取参数（不评估与评估）
params_raw = model.parameters(evaluate=False)
params_eval = model.parameters(evaluate=True)
print("Param count:", len(params_raw))
print("TBR raw/eval:", params_raw.get('TBR'), params_eval.get('TBR'))

Original TBR: 1.1
Updated TBR: 1.15
Restored TBR: 1.1
Param count: 60
TBR raw/eval: 1.1 1.1


In [6]:
# 1.5 Node API：路径导航与属性读写
root = model/None
functions = model/'functions'
physics = model/'physics'
studies = model/'studies'

print("Root exists:", root.exists())
print("Functions children:", [c.name() for c in functions][:5])
print("Physics children:", [c.name() for c in physics][:5])
print("Studies:", [c.name() for c in studies])

# 读取一个研究时间节点的属性（tlist）
time_node = None
for study in studies:
    for feat in study:
        try:
            if feat.java.tag() == 'time':
                time_node = feat
                break
        except Exception:
            pass
    if time_node:
        break

if time_node:
    print("Time node:", time_node)
    print("tlist:", time_node.property('tlist'))
else:
    print("Time node not found.")

Root exists: True
Functions children: ['解析 1']
Physics children: ['全局常微分和微分代数方程']
Studies: ['研究 1']
Time node: studies/研究 1/瞬态
tlist: [       0.    86400.   172800.   259200.   345600.   432000.   518400.
   604800.   691200.   777600.   864000.   950400.  1036800.  1123200.
  1209600.  1296000.  1382400.  1468800.  1555200.  1641600.  1728000.
  1814400.  1900800.  1987200.  2073600.  2160000.  2246400.  2332800.
  2419200.  2505600.  2592000.  2678400.  2764800.  2851200.  2937600.
  3024000.  3110400.  3196800.  3283200.  3369600.  3456000.  3542400.
  3628800.  3715200.  3801600.  3888000.  3974400.  4060800.  4147200.
  4233600.  4320000.  4406400.  4492800.  4579200.  4665600.  4752000.
  4838400.  4924800.  5011200.  5097600.  5184000.  5270400.  5356800.
  5443200.  5529600.  5616000.  5702400.  5788800.  5875200.  5961600.
  6048000.  6134400.  6220800.  6307200.  6393600.  6480000.  6566400.
  6652800.  6739200.  6825600.  6912000.  6998400.  7084800.  7171200.
  7257600.  73

In [7]:
# 1.6 Node.create / remove：临时创建并清理功能节点
tmp_name = 'api_temp_func'
tmp = model/'functions'/tmp_name

if tmp.exists():
    tmp.remove()

try:
    tmp = (model/'functions').create('Analytic', name=tmp_name)
    tmp.property('funcname', 'f_api_test')
    tmp.property('expr', '1')
    tmp.property('fununit', '1')
    print("Created:", tmp, "tag=", tmp.tag())
finally:
    if (model/'functions'/tmp_name).exists():
        (model/'functions'/tmp_name).remove()
        print("Removed temp function node.")

Created: functions/api_temp_func tag= an2
Removed temp function node.


In [8]:
# 1.7 Model.evaluate：结果提取（若已有解）
try:
    # t 为内置时间变量，若解已存在可直接评估
    time_h = model.evaluate('t', unit='h')
    i15 = model.evaluate('I15', unit='g')
    print("Eval OK:", len(time_h), len(i15))
except Exception as e:
    print("Evaluation skipped:", e)

Eval OK: 361 361


In [9]:
# 1.8 清理测试：仅清除结果缓存（不修改模型结构）
try:
    model.clear()
    print("Cleared stored data (plots/solutions/meshes).")
except Exception as e:
    print("Clear skipped:", e)

Cleared stored data (plots/solutions/meshes).


## 2. 进阶补充：树检视、导出与保存

以下内容补充 `tree()` / `inspect()`、导出节点与保存 API 的基本用法。

In [10]:
# 2.1 tree() / inspect()：模型树与节点自省
import mph

# 打印模型树（深度可调）
try:
    mph.tree(model, max_depth=3)
except Exception as e:
    print("tree() skipped:", e)

# 选取一个节点进行 inspect（示例：studies 分组）
try:
    studies = model/'studies'
    mph.inspect(studies)
except Exception as e:
    print("inspect() skipped:", e)

tritum_cycle_Model
├─ parameters
│  ├─ Global Parameters
│  ├─ 参数 2
│  ├─ 参数 3
│  ├─ 参数 5
│  └─ 参数 4
├─ functions
│  └─ 解析 1
├─ components
│  └─ 组件 1
├─ geometries
├─ views
├─ selections
├─ coordinates
├─ variables
│  └─ 变量 1
├─ couplings
├─ physics
│  └─ 全局常微分和微分代数方程
│     └─ 全局方程 1
├─ multiphysics
├─ materials
├─ meshes
├─ studies
│  └─ 研究 1
│     └─ 瞬态
├─ solutions
│  └─ 解 1
│     ├─ 编译方程: 瞬态
│     ├─ 因变量 1
│     └─ 瞬态求解器 1
├─ batches
├─ datasets
│  └─ 研究 1//解 1
├─ evaluations
├─ tables
├─ plots
│  ├─ Full Tritium Inventory Comparison
│  │  └─ 全局 1
│  └─ SDS Tritium Inventory Trend
│     └─ 全局 2
└─ exports
   ├─ 图像 1
   └─ 图像 2
name:    StudyList
tag:     study
display: StudyList
doc:     StudyList
methods:
  clear
  copy
  copyTo
  create
  duplicate
  duplicateTo
  equals
  forEach
  get
  getAPIEngine
  getContainer
  getModelID
  getObjectID
  getObjectLocation
  index
  iterator
  model
  move
  remove
  resolveModelPath
  scope
  size
  spliterator
  tags
  uniquetag


In [13]:
# 2.2 Client.create / remove：创建并清理临时模型
try:
    temp_model = client.create("api_temp_model")
    print("Temp model created:", temp_model.name())
finally:
    try:
        client.remove("api_temp_model")
        print("Temp model removed.")
    except Exception as e:
        print("Temp model cleanup skipped:", e)

Temp model created: api_temp_model
Temp model removed.


In [12]:
# 2.3 Model.export：导出结果（若存在导出节点）
import os

exports = model.exports()
print("Export nodes:", exports)

if exports:
    export_name = exports[0]
    out_path = os.path.join(os.getcwd(), "api_export_test.png")
    try:
        model.export(export_name, file=out_path)
        print("Exported to:", out_path)
    except Exception as e:
        print("Export failed:", e)
else:
    print("No export nodes found; export skipped.")

Export nodes: ['图像 1', '图像 2']
Exported to: d:\Administrator\Documents\Github\tricys_mph\experiment_05\api_export_test.png


In [11]:
# 2.4 Model.save：另存为（避免覆盖原文件）
import os

save_path = os.path.join(os.getcwd(), "api_save_test.mph")
try:
    model.save(save_path)
    print("Saved model to:", save_path)
except Exception as e:
    print("Save skipped:", e)

Saved model to: d:\Administrator\Documents\Github\tricys_mph\experiment_05\api_save_test.mph


## 3. API 总结归纳（速查）

下面按功能域归纳本笔记本涉及的 MPh 主要 API：

### 会话与配置
- `mph.start()`：启动本地会话并返回 `Client`。
- `mph.config.option()`：读取/设置配置项（如 `session`、`caching`、`classkit`）。

### Client（客户端）
- `Client.names()` / `Client.models()`：列出已加载模型。
- `Client.load(path)`：加载 `.mph` 模型文件。
- `Client.create(name)`：创建空模型。
- `Client.remove(model|name)` / `Client.clear()`：移除模型或清空内存模型。
- `Client.modules()` / `Client.cores` / `Client.version`：查询许可模块与会话信息。
- `Client.connect()` / `Client.disconnect()`：连接/断开 Comsol Server（如使用 client–server 模式）。

### Model（模型）
- 结构与信息：`name()`、`file()`、`version()`、`modules()`、`problems()`。
- 求解流程：`build()`、`mesh()`、`solve()`。
- 参数读写：`parameter()`、`parameters()`、`description()`。
- 节点代理：`property()`、`properties()`、`create()`、`remove()`。
- 结果评估：`evaluate()`（支持 `dataset`、`inner`、`outer`）。
- 导入导出：`import_()`、`export()`、`save()`。
- 维护操作：`clear()`、`reset()`。

### Node（节点）
- 导航：`parent()`、`children()`、`exists()`、`name()`、`tag()`、`type()`。
- 属性：`property()`、`properties()`、`comment()`。
- 选择集：`select()`、`selection()`。
- 运行与开关：`run()`、`toggle()`。
- 结构变更：`create()`、`remove()`、`rename()`、`retag()`。

### 工具函数
- `mph.tree(node, max_depth=...)`：打印模型树结构。
- `mph.inspect(node|java)`：打印 Java 节点对象的方法/属性摘要。

> 提示：推荐使用 `tag` 定位节点；在脚本中比 `name` 更稳定。